In [94]:
import  requests, math
import pandas as pd
sisql = lambda arr: requests.get(f"http://penly.cn:7080/kpsi/sisql",params=arr).json() #requests.get("http://penly.cn:7080/kpsi/sisql",params={"sql":"select * from bnc where s like 'work:VERB:VB%'  or s in ('VERB:work')", "divby":"VERB:work"}).json()
rows = lambda sql, row_idx=None: sisql({"sql":sql}) if row_idx is None else [row[row_idx] for row in sisql({"sql":sql})]  # rows("select * from bnc limit 2") => [['#snt', 5311102], ['LEX:it', 1006089]]
getsi = lambda s , corpus='dic', defa=0.1:  ( res:=rows(f"select i from {corpus} where s = '{s}'"),  res[0][0] if len(res) > 0 else defa  )[1]
keyness = lambda a,b,c,d : round(a * math.log(a*(c+d)/(c*(a+b))) +    b * math.log(b*(c+d)/(d*(a+b)))  if a/c > b/d else  -  a * math.log(a*(c+d)/(c*(a+b)))  -  b * math.log(b*(c+d)/(d*(a+b))), 1)
pos_attr = lambda pos='VERB': [ row[0] for row in  rows(f"select attr from pos_attr where pos  = '{pos}'")]
wordlist = lambda pos='VERB': [ row[0] for row in  rows(f"select substring_index(k,':',-1) from wordlist where k  like '{pos}:%'")]
kn = lambda s1, s2, cp1, cp2:  keyness( getsi(s1, cp1), getsi(s1, cp2), getsi(s2,cp1), getsi(s2, cp2) )
getlist = lambda pat="~dobj_VERB_NOUN:knowledge:%" , cp='dic':  [ row[0] for row in  rows(f"select substring_index(s,':',-1) from {cp} where s like '{pat}'")]
getunion = lambda pat="~dobj_VERB_NOUN:knowledge:%" , cp1='dic', cp2='clec':  [ row[0] for row in  rows(f"select substring_index(s,':',-1) from {cp1} where s like '{pat}' union select substring_index(s,':',-1) from {cp2} where s like '{pat}'")]
getmap = lambda pat="~dobj_VERB_NOUN:knowledge:%" , cp='dic':  dict(rows(f"select substring_index(s,':',-1), i from {cp} where s like '{pat}'"))

## corpus overview

In [81]:
(sisql({"sql":"select * from dic where s in ('#LEX','#LEM','#snt','#VERB','#NOUN','#ADJ','#CC','#ADV','#MD')", "divby":"#LEX"}),
 sisql({"sql":"select * from bnc where s in ('#LEX','#LEM','#snt','#VERB','#NOUN','#ADJ','#CC','#ADV','#MD')", "divby":"#LEX"}),
 kn("#ADV","#LEX","dic","bnc"),
pd.DataFrame(sisql({"sql":"select * from bnc where s like '#VERB:%'  or s in ('#VERB')", 'divby':'#VERB'}), columns=['attr','freq','perc']) #平均动词
)

([['#ADJ', 764283, 0.0771],
  ['#ADV', 438518, 0.0443],
  ['#LEM', 8666929, 0.8748],
  ['#LEX', 9907790, 1.0],
  ['#NOUN', 2078451, 0.2098],
  ['#VERB', 1212990, 0.1224],
  ['#snt', 681946, 0.0688]],
 [['#ADJ', 7076709, 0.0656],
  ['#ADV', 4876730, 0.0452],
  ['#LEM', 91225689, 0.8452],
  ['#LEX', 107935228, 1.0],
  ['#NOUN', 18882238, 0.1749],
  ['#VERB', 11840022, 0.1097],
  ['#snt', 5311102, 0.0492]],
 -86.0,
                attr      freq    perc
 0             #VERB  11840022  1.0000
 1          #VERB:MD   1381788  0.1167
 2          #VERB:VB   2684891  0.2268
 3         #VERB:VBD   2065551  0.1745
 4         #VERB:VBG   1667798  0.1409
 ..              ...       ...     ...
 89      #VERB:~prep     67059  0.0057
 90       #VERB:~prt        23  0.0000
 91  #VERB:~quantmod        29  0.0000
 92     #VERB:~relcl    803595  0.0679
 93     #VERB:~xcomp    909717  0.0768
 
 [94 rows x 3 columns])

In [70]:
# forget VBG 少用， work VBG 超用
kn_attr = lambda a, b, c, d, cp='bnc': ( res:= dict(rows(f"select * from {cp} where s in ('{a}', '{b}', '{c}', '{d}')")), keyness( res.get(a,0.1), res.get(b,0.1), res.get(c,0.1),res.get(d,0.1) ) )
w = 'work'
kn_attr(f'{w}:VERB:VBG','#VERB:VBG',f'VERB:{w}','#VERB')

({'#VERB': 11840022,
  '#VERB:VBG': 1667798,
  'VERB:work': 64492,
  'work:VERB:VBG': 24392},
 8714.6)

In [72]:
# /kpsi/attr_keyness_of_two_corpus'
(w:="work", attr:="VBG", pos:="VERB", cp1:="sino", cp2:="bnc",  kn(f'{w}:{pos}:{attr}', f'{pos}:{w}', cp1, cp2))

('work', 'VBG', 'VERB', 'sino', 'bnc', -476.5)

## verb portal

In [73]:
# pos attr
(w:="consider", pos:="VERB", cp1:="sino", cp2:="bnc", ascending:=True)
df = pd.DataFrame([ (attr, kn(f'{w}:{pos}:{attr}', '{pos}:{w}', cp1,cp2)) for attr in pos_attr(pos) ], columns=['attr','kn'])
df.sort_values(by="kn" , ascending=ascending)

,attr,kn
4,aux,-1204.8
31,VBN,-1161.0
34,vp,-1032.4
5,auxpass,-640.1
27,svo,-600.5
14,dobj,-525.2
6,be_vbn_p,-520.5
29,VBD,-513.9
22,oprd,-506.6
52,~xcomp,-460.2


In [69]:
# triple_keyness_of_two_corpus
(pat:="~dobj_VERB_NOUN:knowledge", cp1:="sino", cp2:='bnc', ascending:=True)
words = getunion(f"{pat}:%",cp1, cp2)
clec = getmap(f"{pat}:%",cp1)
dic = getmap(f"{pat}:%", cp2)
c = getsi(f"#{pat}", cp1)
d = getsi(f"#{pat}", cp2)    
df = pd.DataFrame({'words': words, cp1: [ clec.get(w, 0) for w in words], cp2: [ dic.get(w, 0) for w in words], 'keyness':[ keyness(clec.get(w,0.1), dic.get(w,0.1), c, d)  for w in words]})
df.sort_values(by="keyness" , ascending=ascending)

,words,sino,bnc,keyness
179,deny,2,58,-129.6
535,require,98,108,-124.3
184,develop,43,47,-53.8
299,have,90,62,-53.7
250,extend,43,43,-47.0
...,...,...,...,...
362,know,523,0,49.9
603,study,601,0,57.5
620,teach,810,5,59.9
224,enrich,870,0,83.5


In [65]:
# chunk_keyness_of_two_corpus
(w:="book", pos:='NOUN', seg:='np', cp1:="clec", cp2:='dic') 
words = getunion(f"{w}:{seg}:%",cp1,cp2)
clec = getmap(f"{w}:{seg}:%",cp1)
dic = getmap(f"{w}:{seg}:%",cp2)
c = getsi(f"{w}:{pos}:{seg}", cp1)
d = getsi(f"{w}:{pos}:{seg}", cp2)
df = pd.DataFrame({'word': words, cp1: [ clec.get(w, 0) for w in words], cp2: [ dic.get(w, 0) for w in words], 'keyness':[ keyness(clec.get(w,0.1), dic.get(w,0.1), c, d)  for w in words]})
df.sort_values(by="keyness" , ascending=True)

,word,clec,dic,keyness
144,the book,74,1089,-25.7
178,this book,14,294,-12.0
541,her book,0,63,-8.1
58,his book,2,97,-7.6
542,her books,0,40,-5.0
...,...,...,...,...
80,more books,13,4,18.0
168,the text book,11,0,22.1
145,the books,68,160,22.9
72,many books,19,6,26.1


In [71]:
# pos_attr_keyness_of_one_corpus
(attr:="VBG", pos:="VERB", cp:="bnc", ascending:=True)
b = getsi(f"#{pos}:{attr}", cp)
d = getsi(f"#{pos}", cp)    
df = pd.DataFrame([ (w,  keyness( getsi(f'{w}:{pos}:{attr}', cp), b, getsi(f"{pos}:{w}",cp), d ) )  for w in wordlist(pos) if w.isalpha() ], columns=['word', 'kn'])
df.sort_values(by="kn" , ascending=ascending)    

,word,kn
5637,will,-31682.4
674,can,-29890.1
2836,know,-11744.1
4413,say,-10013.3
4493,see,-9497.4
...,...,...
2109,follow,9451.0
2296,go,11494.7
1587,do,12105.2
40,accord,15763.9


In [ ]:
# 动词 consider 的属性 超用 少用 对比，  VBN 少用， ccomp 超用
w,pos,cp1,cp2 = 'consider','VERB','sino','bnc'
df = pd.DataFrame([ (attr, kn(f'{w}:{pos}:{attr}', '{pos}:{w}', cp1,cp2)) for attr in pos_attr(pos) ], columns=['attr','kn'])
df.sort_values(by="kn" , ascending=True)

In [34]:
rows("select * from bnc where s in ('belong:VERB:VBG', '#VERB:VBG', 'VERB:belong', '#VERB')")

[['#VERB', 11840022],
 ['#VERB:VBG', 1667798],
 ['VERB:belong', 5819],
 ['belong:VERB:VBG', 1249]]

In [16]:
#动词词性细分饼图
sisql({"sql":"select * from bnc where s like 'work:VERB:VB%'  or s in ('VERB:work')", "divby":"VERB:work"})

[['VERB:work', 64492, 1.0],
 ['work:VERB:VB', 18662, 0.2894],
 ['work:VERB:VBD', 7358, 0.1141],
 ['work:VERB:VBG', 24392, 0.3782],
 ['work:VERB:VBN', 4891, 0.0758],
 ['work:VERB:VBP', 4191, 0.065],
 ['work:VERB:VBZ', 4998, 0.0775]]

In [29]:
rows("select * from dic where s like 'work:VERB:VB%'  or s in ('VERB:work')")

[['VERB:work', 7070],
 ['work:VERB:VB', 1978],
 ['work:VERB:VBD', 988],
 ['work:VERB:VBG', 2430],
 ['work:VERB:VBN', 421],
 ['work:VERB:VBP', 538],
 ['work:VERB:VBZ', 715]]

In [24]:
#平均动词
pd.DataFrame(sisql({"sql":"select * from bnc where s like '#VERB:%'  or s in ('#VERB')", 'divby':'#VERB'}), columns=['attr','freq','perc'])
#df.sort_values(by="perc")

,attr,freq,perc
0,#VERB,11840022,1.0000
1,#VERB:MD,1381788,0.1167
2,#VERB:VB,2684891,0.2268
3,#VERB:VBD,2065551,0.1745
4,#VERB:VBG,1667798,0.1409
...,...,...,...
89,#VERB:~prep,67059,0.0057
90,#VERB:~prt,23,0.0000
91,#VERB:~quantmod,29,0.0000
92,#VERB:~relcl,803595,0.0679


In [68]:
# scale_keyness
from cikuu.dic.lemma_scale import lemma_scale
(scale_beg:=5.0, scale_end:=6.0, cp1:="sino", cp2:="bnc", ascending:=True)
df = pd.DataFrame([ (w, kn(f'LEM:{w}', '#LEM', cp1,cp2)) for w, s in lemma_scale.items() if s >= scale_beg and s <= scale_end ], columns=['attr','kn'])
df.sort_values(by="kn" , ascending=ascending)

,attr,kn
2,pre,-2323.3
606,rugby,-810.5
911,constable,-522.0
188,borough,-501.9
956,onwards,-469.7
...,...,...
769,shortcoming,1403.1
34,tuition,1623.8
329,indispensable,1789.2
470,cherish,2331.2


In [83]:
rows("select * from bnc where s like '~dobj_VERB_NOUN:knowledge:%' order by i desc limit 12")

[['~dobj_VERB_NOUN:knowledge:use', 160],
 ['~dobj_VERB_NOUN:knowledge:acquire', 127],
 ['~dobj_VERB_NOUN:knowledge:require', 108],
 ['~dobj_VERB_NOUN:knowledge:have', 62],
 ['~dobj_VERB_NOUN:knowledge:gain', 60],
 ['~dobj_VERB_NOUN:knowledge:deny', 58],
 ['~dobj_VERB_NOUN:knowledge:give', 54],
 ['~dobj_VERB_NOUN:knowledge:increase', 48],
 ['~dobj_VERB_NOUN:knowledge:share', 47],
 ['~dobj_VERB_NOUN:knowledge:develop', 47],
 ['~dobj_VERB_NOUN:knowledge:provide', 45],
 ['~dobj_VERB_NOUN:knowledge:extend', 43]]

In [110]:
{ "bnc": sisql({"sql":"select * from bnc where s in ('ADV:utterly', 'ADV:completely','ADV:totally','#snt')","divby":"#snt"}),
"nju": sisql({"sql":"select * from nju where s in ('ADV:utterly', 'ADV:completely','ADV:totally','#snt')","divby":"#snt"}) }

{'bnc': [['#snt', 5311102, 1.0],
  ['ADV:completely', 7987, 0.0015038310316766652],
  ['ADV:totally', 5511, 0.0010376377633116443],
  ['ADV:utterly', 1213, 0.0002283895131368217]],
 'nju': [['#snt', 2068479, 1.0],
  ['ADV:completely', 4846, 0.0023427842390471455],
  ['ADV:totally', 6425, 0.0031061470771518588],
  ['ADV:utterly', 264, 0.00012763001219736822]]}

In [93]:
kn("ADV:utterly", "#ADV", 'sino', 'bnc')

-285.1

In [108]:
clec= dict(rows("select substring_index(s,':',-1), i from clec where s like 'VERB:%'"))
[ (k,v) for k,v in rows("select substring_index(s,':',-1),i from bnc where s like 'VERB:%' order by i desc") if not k in clec and k.isalpha()] 

[('register', 4096),
 ('incorporate', 3931),
 ('ai', 3446),
 ('integrate', 3149),
 ('comprise', 3078),
 ('tackle', 2894),
 ('suspend', 2747),
 ('knit', 2557),
 ('highlight', 2518),
 ('erm', 2312),
 ('breed', 2201),
 ('evolve', 2091),
 ('precede', 2078),
 ('rub', 1993),
 ('colour', 1975),
 ('undermine', 1898),
 ('label', 1849),
 ('contrast', 1791),
 ('shed', 1725),
 ('envisage', 1701),
 ('opt', 1689),
 ('update', 1678),
 ('commission', 1646),
 ('subject', 1542),
 ('target', 1530),
 ('amend', 1528),
 ('signal', 1484),
 ('tip', 1452),
 ('edit', 1373),
 ('revive', 1373),
 ('dispose', 1365),
 ('depict', 1326),
 ('sack', 1296),
 ('characterise', 1272),
 ('er', 1269),
 ('activate', 1258),
 ('compile', 1244),
 ('market', 1240),
 ('grip', 1230),
 ('fuck', 1219),
 ('curl', 1198),
 ('entail', 1188),
 ('tuck', 1185),
 ('presume', 1170),
 ('document', 1151),
 ('alright', 1137),
 ('nominate', 1131),
 ('position', 1119),
 ('term', 1119),
 ('divert', 1114),
 ('underline', 1114),
 ('fling', 1102),
 ('d